<a href="https://colab.research.google.com/github/Cleander/analise-de-sentimentos/blob/main/analise_de_sentimentos_bertimbal_pi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets torch pandas

In [ ]:
import pandas as pd

df = pd.read_csv('olist.csv')

df.head()

In [ ]:
df.isnull().sum()

In [ ]:
print(f"Linhas antes da limpeza: {df.shape[0]}")
df = df.dropna(subset=['review_text_tokenized', 'polarity'])
print(f"Linhas após a limpeza: {df.shape[0]}")

In [ ]:
train_data = df[(df['kfold_polarity'] >= 2) & (df['kfold_polarity'] <= 8)]
val_data = df[df['kfold_polarity'] == 9]
test_data = df[df['kfold_polarity'] == 1]

print(f"Treinamento: {len(train_data)}")
print(f"Validação: {len(val_data)}")
print(f"Teste: {len(test_data)}")

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128)

train_encodings = tokenize_function(train_data['review_text_tokenized'].tolist())
val_encodings = tokenize_function(val_data['review_text_tokenized'].tolist())
test_encodings = tokenize_function(test_data['review_text_tokenized'].tolist())


In [ ]:
import torch
from datasets import Dataset

train_labels = torch.tensor(train_data['polarity'].values, dtype=torch.long)
val_labels = torch.tensor(val_data['polarity'].values, dtype=torch.long)
test_labels = torch.tensor(test_data['polarity'].values, dtype=torch.long)

train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})


In [ ]:
from transformers import BertForSequenceClassification

# Classificação binária (polaridade)
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=2)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    fp16=True,
)

//Revisar parâmetros
//Biblioteca Tune

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import torch

def compute_metrics(dataset):
    predictions = trainer.predict(dataset)
    preds = torch.argmax(torch.tensor(predictions.predictions), axis=1)
    labels = dataset["labels"]
    accuracy = accuracy_score(labels, preds)
    report = classification_report(labels, preds, target_names=["Negativo", "Positivo"])

    print(f"Acurácia: {accuracy:.4f}")
    print("Relatório de Classificação:\n", report)

compute_metrics(test_dataset)

In [ ]:
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model.save_pretrained("/content/drive/MyDrive/sentiment_model")
tokenizer.save_pretrained("/content/drive/MyDrive/sentiment_model")

In [ ]:
model = BertForSequenceClassification.from_pretrained("./sentiment_model")
tokenizer = BertTokenizer.from_pretrained("./sentiment_model")

def predict_sentiment(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
    outputs = model(**encodings)
    predictions = outputs.logits.argmax(dim=-1)
    return predictions

textos = ["Este úlitmo lançamento não foi legal", "Não podia ter comprado um produto melhor."]
predictions = predict_sentiment(textos)
print(predictions)

##Utilizando o Modelo e a API

In [ ]:
pip install fpdf

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from fpdf import FPDF
from datetime import datetime
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ig_user_id = '17841473759317909'
app_id = "1011495597278535"
app_secret = "9bafc75955fe398c1e6ee94a4464bcf0"
user_access_token = "EAAOX8yvEhUcBO5drL5D0Doaaeuw13B16bTOPyK0115YK0w0uvi2vJ49otgg6ykqkf5eoAACSqZBzVA6YmmGdp8ZBNuNkKo4lDiCihrjZABtIUZAxccXnC8GvHRrzuvINwzt1yVMdMZAZB4O9zhC5qSJl6WCQG5ZBeptNoWBjsErEOqW1BCN67optJfTVccN4oyvEAZDZD"

url = f"https://graph.facebook.com/v17.0/oauth/access_token?grant_type=fb_exchange_token&client_id={app_id}&client_secret={app_secret}&fb_exchange_token={user_access_token}"
response = requests.get(url)
long_access_token = response.json()["access_token"]

base_url = f"https://graph.facebook.com/v17.0/{ig_user_id}/media?fields=id,caption,timestamp&access_token={long_access_token}"

if not os.path.exists('graficos'):
    os.makedirs('graficos')

In [ ]:
def coletar_comentarios_por_publicacao():
    publicacoes = []
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()['data']
        for item in data:
            media_id = item['id']
            caption = item.get('caption', 'Sem legenda')
            timestamp = item.get('timestamp', None)

            comments_url = f'https://graph.facebook.com/v17.0/{media_id}/comments?fields=id,text,timestamp,username&access_token={long_access_token}'
            comments_response = requests.get(comments_url)

            comentarios = []
            if comments_response.status_code == 200:
                comments_data = comments_response.json().get('data', [])
                comentarios = [comment['text'] for comment in comments_data]
            else:
                print(f'Erro ao buscar comentários da mídia {media_id}')

            publicacoes.append({
                'media_id': media_id,
                'caption': caption,
                'comentarios': comentarios,
                'timestamp': timestamp
            })
    else:
        print('Erro ao buscar mídias:', response.text)

    return publicacoes

In [ ]:
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/sentiment_model")
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/sentiment_model")

In [ ]:
def analisar_sentimentos(comentarios):
    resultados = []
    if comentarios:
        encodings = tokenizer(comentarios, padding=True, truncation=True, max_length=128, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**encodings)
            predictions = outputs.logits.argmax(dim=-1)

        for comentario, pred in zip(comentarios, predictions):
            sentimento = 'Positivo' if pred.item() == 1 else 'Negativo'
            resultados.append((comentario, sentimento))
    return resultados

In [ ]:
def calcular_metricas(resultados):
    total = len(resultados)
    positivos = sum(1 for _, s in resultados if s == 'Positivo')
    negativos = total - positivos
    porcentagem_positivos = positivos / total * 100 if total else 0
    porcentagem_negativos = negativos / total * 100 if total else 0
    return positivos, negativos, porcentagem_positivos, porcentagem_negativos

In [ ]:
def gerar_grafico_publicacao(caption, positivos, negativos, media_id):
    labels = ['Positivos', 'Negativos']
    sizes = [positivos, negativos]
    colors = ['#4CAF50', '#F44336']

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
    ax.axis('equal')
    plt.title(caption[:50] + '...')
    caminho = f'graficos/{media_id}.png'
    plt.savefig(caminho)
    plt.close()
    return caminho

"""
def gerar_grafico_geral(total_positivos, total_negativos):
    labels = ['Positivos', 'Negativos']
    sizes = [total_positivos, total_negativos]
    colors = ['#4CAF50', '#F44336']

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
    ax.axis('equal')
    plt.title('Distribuição Geral dos Sentimentos')
    caminho = 'graficos/geral.png'
    plt.savefig(caminho)
    plt.close()
    return caminho
"""

def gerar_grafico_temporal(resultados_temporais):
    if not resultados_temporais:
        return None

    resultados_ordenados = sorted(resultados_temporais, key=lambda x: x['data'])

    datas = [
      datetime.strptime(item['data'], '%Y-%m-%dT%H:%M:%S%z').strftime('%d/%m/%Y %H:%M')
      for item in resultados_ordenados
    ]

    porcentagens = [item['pct_positivos'] for item in resultados_ordenados]
    legends = [item['caption'][:30] + '...' if len(item['caption']) > 30 else item['caption'] for item in resultados_ordenados]

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(datas, porcentagens, marker='o', color='#2196F3', linestyle='-')

    for i, txt in enumerate(legends):
        ax.annotate(txt, (i, porcentagens[i]), textcoords="offset points", xytext=(0,10),
                    ha='center', fontsize=8, rotation=45)

    ax.set_xticks(datas)
    ax.set_xticklabels(datas, rotation=45, ha='right', fontsize=8)

    ax.set_title('Evolução da Avaliação das Publicações ao Longo do Tempo')
    ax.set_xlabel('Data da Publicação')
    ax.set_ylabel('% de Comentários Positivos')
    ax.set_ylim(0, 100)
    ax.grid(True)

    caminho = 'graficos/grafico_temporal.png'
    plt.tight_layout()
    plt.savefig(caminho)
    plt.close()
    return caminho


In [ ]:
class PDFRelatorio(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 16)
        self.cell(0, 10, 'Relatório de Análise de Sentimentos - Instagram', 0, 1, 'C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    def add_publicacao(self, caption, positivos, negativos, porcentagem_positivos, porcentagem_negativos, grafico_path):
        self.set_font('Arial', 'B', 12)
        self.multi_cell(0, 10, f'Publicação: {caption[:100]}...')
        self.set_font('Arial', '', 12)
        self.cell(0, 10, f'Positivos: {positivos} ({porcentagem_positivos:.2f}%)', 0, 1)
        self.cell(0, 10, f'Negativos: {negativos} ({porcentagem_negativos:.2f}%)', 0, 1)
        self.ln(3)
        self.image(grafico_path, w=150)
        self.ln(10)

    def add_conclusao_geral(self, total_positivos, total_negativos, pct_positivos, pct_negativos, grafico_path):
        self.add_page()
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, 'Resumo Geral', 0, 1, 'C')
        self.ln(5)
        self.set_font('Arial', '', 12)
        self.cell(0, 10, f'Total de Comentários Positivos: {total_positivos} ({pct_positivos:.2f}%)', 0, 1)
        self.cell(0, 10, f'Total de Comentários Negativos: {total_negativos} ({pct_negativos:.2f}%)', 0, 1)
        self.ln(5)
        self.image(grafico_path, w=150)
        self.ln(10)

        conclusao = 'Conclusão geral: '
        if pct_positivos > 70:
            conclusao += 'O perfil está muito bem avaliado!'
        elif pct_positivos > 40:
            conclusao += 'O perfil está com avaliação mista.'
        else:
            conclusao += 'O perfil está sendo mal avaliado.'

        self.multi_cell(0, 10, conclusao)

In [ ]:
publicacoes = coletar_comentarios_por_publicacao()
pdf = PDFRelatorio()
pdf.add_page()

print(publicacoes)

total_resultados = []
resultados_temporais = []

total_positivos = 0
total_negativos = 0

for publicacao in publicacoes:
    comentarios = publicacao['comentarios']
    caption = publicacao['caption']
    media_id = publicacao['media_id']

    if comentarios:
        resultados = analisar_sentimentos(comentarios)
        positivos, negativos, pct_positivos, pct_negativos = calcular_metricas(resultados)
        grafico_path = gerar_grafico_publicacao(caption, positivos, negativos, media_id)

        pdf.add_publicacao(caption, positivos, negativos, pct_positivos, pct_negativos, grafico_path)

        total_positivos += positivos
        total_negativos += negativos
        total_resultados.extend(resultados)

        resultados_temporais.append({
            'data': publicacao['timestamp'],
            'pct_positivos': pct_positivos,
            'caption': caption
        })

grafico_temporal_path = gerar_grafico_temporal(resultados_temporais)
pct_total_positivos = total_positivos / (total_positivos + total_negativos) * 100 if (total_positivos + total_negativos) else 0
pct_total_negativos = 100 - pct_total_positivos

pdf.add_conclusao_geral(total_positivos, total_negativos, pct_total_positivos, pct_total_negativos, grafico_temporal_path)

pdf.output('relatorio_sentimentos_instagram.pdf')
